# Zadatak 3 - iskazna logika, DPLL

Klasa za zapis formule. 

Formula moze imati argumente i to bi bili operandi koji ucestvuju u formuli. 
Na primer, za konjukciju postoje dva operanda.

In [8]:
class Formula():
    def __init__(self, args=None):
        self.args = args
        
class T(Formula):
    def __init__(self):
        super().__init__()
        
    def __str__(self):
        return 'T'
    
    def interpretation(self, valuation):
        return True
    
class F(Formula):
    def __init__(self):
        super().__init__()
        
    def __str__(self):
        return 'F'
    
    def interpretation(self, valuation):
        return False
    
class Letter(Formula):
    def __init__(self, character):
        super().__init__()
        self.character = character
        
    def __str__(self):
        return self.character
    
    def interpretation(self, valuation):
        return valuation[self.character]

class Not(Formula):
    def __init__(self, operand):
        super().__init__([operand])
        
    def __str__(self):
        return '~({})'.format(self.args[0])
    
    def interpretation(self, valuation):
        return not self.args[0].interpretation(valuation)
    
class And(Formula):
    def __init__(self, operand1, operand2):
        super().__init__([operand1, operand2])
        
    def __str__(self):
        return '(({}) & ({}))'.format(self.args[0], self.args[1])
    
    def interpretation(self, valuation):
        return self.args[0].interpretation(valuation) and self.args[1].interpretation(valuation)
    
class Or(Formula):
    def __init__(self, operand1, operand2):
        super().__init__([operand1, operand2])
        
    def __str__(self):
        return '(({}) | ({}))'.format(self.args[0], self.args[1])
    
    def interpretation(self, valuation):
        return self.args[0].interpretation(valuation) or self.args[1].interpretation(valuation)
    
class Imp(Formula):
    def __init__(self, operand1, operand2):
        super().__init__([operand1, operand2])
        
    def __str__(self):
        return '(({}) => ({}))'.format(self.args[0], self.args[1])
    
    def interpretation(self, valuation):
        return not self.args[0].interpretation(valuation) or self.args[1].interpretation(valuation)
    
class Eq(Formula):
    def __init__(self, operand1, operand2):
        super().__init__([operand1, operand2])
        
    def __str__(self):
        return '(({}) <=> ({}))'.format(self.args[0], self.args[1])
    
    def interpretation(self, valuation):
        return self.args[0].interpretation(valuation) == self.args[1].interpretation(valuation)

Funkcija za odredjivanje konjuktivne normalne forme za datu formulu.

In [11]:
def CNF(formula):
    if isinstance(formula, T) or isinstance(formula, F) or isinstance(formula, Letter):
        return formula
    
    if isinstance(formula, Eq):
        [A, B] = formula.args
        return CNF(And(
            Imp(A, B),
            Imp(B, A)
        ))
    
    if isinstance(formula, Imp):
        [A, B] = formula.args
        return CNF(Or(
            Not(A),
            B
        ))
    
    if isinstance(formula, Not):
        argF = CNF(formula.args[0])
        
        if isinstance(argF, And):
            [A, B] = argF.args
            return CNF(Or(
                Not(A),
                Not(B)
            ))
        
        if isinstance(argF, Or):
            [A, B] = argF.args
            return CNF(And(
                Not(A),
                Not(B)
            ))
        
        if isinstance(argF, Not):
            return CNF(argF.args[0])
        
        return Not(argF)
        
    if isinstance(formula, Or):
        [A, B] = formula.args
        
        A = CNF(A)
        B = CNF(B)
        
        if isinstance(A, And):
            return CNF(And(
                    Or(A.args[0], B),
                    Or(A.args[1], B)
                ))
            
        
        if isinstance(B, And):
            return CNF(And(
                    Or(A, B.args[0]),
                    Or(A, B.args[1])
                ))
            
        return Or(
            A,
            B
        )
    
    if isinstance(formula, And):
        return And(
            CNF(formula.args[0]),
            CNF(formula.args[1])
        )

Funkcija _dimacs_ formulu iz KNF oblika prevodi u dimacs string format.

In [13]:
def get_literals(clause):
    if isinstance(clause, Or):
        return get_literals(clause.args[0]) + get_literals(clause.args[1])
    
    else:
        return [clause]

def get_clauses(cnf_formula):
    if isinstance(cnf_formula, And):
        return get_clauses(cnf_formula.args[0]) + get_clauses(cnf_formula.args[1])
    
    else:
        return [get_literals(cnf_formula)]
    
def all_letters(formula):
    if isinstance(formula, Letter):
        return [formula.character]
    
    else:
        if len(formula.args) == 1:
            return all_letters(formula.args[0])
        elif len(formula.args) == 2:
            return list(set(all_letters(formula.args[0]) + all_letters(formula.args[1])))
        else:
            return []

def _dimacs(formula):
    formula_cnf = CNF(formula)
    
    letters = all_letters(formula_cnf)
    
    mapping = {}
    
    for (index, letter) in enumerate(letters):
        mapping[letter] = index + 1
        
    clauses = get_clauses(formula_cnf)
    
    num_clauses = len(clauses)
    num_letters = len(letters)
    
    header = 'p cnf {} {}\n'.format(num_letters, num_clauses)
    
    body = ''
    
    for clause in clauses:
        line = ''
        
        for literal in clause:
            if isinstance(literal, Letter):
                line += str(mapping[literal.character]) + ' '
            else:
                line += str(- mapping[literal.args[0].character]) + ' '
                
        line += '0\n'
        body += line
        
    return header + body    

def dimacs(formula):
    dimacs_formula = _dimacs(formula)
    
    lines = dimacs_formula.split('\n')
    
    D = []
    
    for line in lines:
        if len(line) == 0 or line[0] == 'c' or line[0] == 'p':
            continue;
            
        clause = []
        
        literals = [int(l) for l in line.split(' ')]
        
        for literal in literals:
            if literal != 0:
                clause.append(literal)
                
        D.append(clause)
        
    return D

Formula se zadaje u dimacs formatu kao lista listi. 

Funkcija _DPLL_ prima listu listi i praznu valuaciju. Vraca DA i valuaciju ukoliko je formula zadovoljiva, a NE ukoliko nije.

a) Dopuniti funkciju DPLL implementacijom pravila **pure literal**.

In [64]:
import copy

def replace_literal(D, literal, value):
    num_clauses = len(D)
    D_copy = copy.copy(D)
    for i in range(num_clauses):
        D_copy[i] = [value if l == literal else l for l in D_copy[i]]
        D_copy[i] = ['-{}'.format(value) if l == -literal else l for l in D_copy[i]]
        
    return D_copy

# Formula D je u dimacs formatu, kao lista listi
def DPLL(D, valuation, ispis):
    num_clauses = len(D)
    
    if num_clauses == 0:
        return True, valuation
    
    # ---------------------------------------------------
    # {NEGATION SUPSTITION}
    # ---------------------------------------------------
    new_D = []
    has_changes = False
    for i in range(num_clauses):
        clause = D[i]
        m = len(clause)
        
        if m == 0:
            return False, None
        
        for j in range(m):
            if clause[j] == '-T':
                clause[j] = 'F'
                has_changes = True
            
            elif clause[j] == '-F':
                clause[j] = 'T'
                has_changes = True
        
        new_D.append(clause)
        
    D = copy.copy(new_D)
    
    if has_changes:
        if ispis:
            print('NEGATION SUBSTITUTION:')
            print(D)
        
    new_D = []
    has_changes = False
    for clause in D:
        new_clause = []
        for l in clause:
            if l != 'F':
                new_clause.append(l)
            else:
                has_changes = True
        new_D.append(new_clause)
    
    if has_changes:
        D = copy.copy(new_D)
        if ispis:
            print("DELETED F from formula")
            print(D)

    for clause in D:
        if len(clause) == 0:
            return False, None
        
    # ---------------------------------------------------
    # {TAUTOLOGY}
    # ---------------------------------------------------
    new_D = []
    has_changes = False
       
    for clause in D:
        new_clause = []
        keep_clause = True
        
        for literal in clause:
            if literal == 'T':
                keep_clause = False
                has_changes = True
                break
                
            if literal != 'F':
                if -literal in clause:
                    # Postoji literal i njegova negacija u klauzi
                    keep_clause = False
                    has_changes = True
                    break
                else:
                    # Inace zadrzavamo literal
                    new_clause.append(literal)
            else:
                has_changes = True
                
        if keep_clause:
            new_D.append(new_clause)
            
    D = copy.copy(new_D)
    if has_changes:
        if ispis:
            print('TAUTOLOGY:')
            print(D)
        return DPLL(D, valuation, ispis)
    
    # ---------------------------------------------------
    # {UNIT PROPAGATION}
    # ---------------------------------------------------
    new_D = []
    has_changes = False
    for clause in D:
        new_clause = []
        keep_clause = True
        m = len(clause)
        if m == 1:
            # Jedinicna klauza
            has_changes = True
            l = clause[0]
            if l > 0:
                valuation[l] = True
                new_D = replace_literal(D, abs(l), 'T')
            else:
                valuation[-l] = False
                new_D = replace_literal(D, abs(l), 'F')
            break
        if keep_clause:
            new_D.append(clause)
    D = copy.copy(new_D)
    if has_changes:
        if ispis:
            print('UNIT PROPAGATION')
            print(D)
        return DPLL(D, valuation, ispis)
    
    # ---------------------------------------------------
    # {PURE LITERAL}
    # ---------------------------------------------------
    new_D = []
    has_changes = False
    all_literals = []
    for clause in D:
        all_literals += clause
        
    distinct_literals = list(set([l for l in all_literals]))
    if ispis:
        print(distinct_literals)
    for l in distinct_literals:
        if -l not in all_literals:
            has_changes = True
            if l > 0:
                valuation[l] = True
                new_D = replace_literal(D, abs(l), 'T')
            else:
                valuation[-l] = False
                new_D = replace_literal(D, abs(l), 'F')
            break
            
    if has_changes:
        D = copy.copy(new_D)
        if ispis:
            print('PURE LITERAL:')
            print(D)
        return DPLL(D, valuation, ispis)
    
    # ---------------------------------------------------
    # {SPLIT}
    # ---------------------------------------------------
    new_D = []
    selected_literal = distinct_literals[0]
    valuation[selected_literal] = True
    new_D = replace_literal(D, selected_literal, 'T')
    if ispis:
        print('SPLIT [{}->T]:'.format(selected_literal))
        print(new_D)
    res, valuation2 = DPLL(new_D, valuation, ispis)
    
    if res:
        return True, valuation2
    
    valuation[selected_literal] = False
    new_D = replace_literal(D, selected_literal, 'F')
    
    if ispis:
        print('SPLIT [{}->F]:'.format(selected_literal))
        print(new_D)
    return DPLL(new_D, valuation, ispis)

In [65]:
test_D = [[-3, 4],[-1, 2, -3],[3,1,-2],['-T', '-F']]
DPLL(test_D, {}, True)

NEGATION SUBSTITUTION:
[[-3, 4], [-1, 2, -3], [3, 1, -2], ['F', 'T']]
DELETED F from formula
[[-3, 4], [-1, 2, -3], [3, 1, -2], ['T']]
TAUTOLOGY:
[[-3, 4], [-1, 2, -3], [3, 1, -2]]
[1, 2, 3, 4, -2, -3, -1]
PURE LITERAL:
[[-3, 'T'], [-1, 2, -3], [3, 1, -2]]
TAUTOLOGY:
[[-1, 2, -3], [3, 1, -2]]
[1, 2, 3, -2, -3, -1]
SPLIT [1->T]:
[['-T', 2, -3], [3, 'T', -2]]
NEGATION SUBSTITUTION:
[['F', 2, -3], [3, 'T', -2]]
DELETED F from formula
[[2, -3], [3, 'T', -2]]
TAUTOLOGY:
[[2, -3]]
[2, -3]
PURE LITERAL:
[['T', -3]]
TAUTOLOGY:
[]


(True, {4: True, 1: True, 2: True})

b) Testirati rad DPLL nad formulom:
   $(p \Leftrightarrow q) \Rightarrow (\neg p \land r)$

In [70]:
p = Letter('p')
q = Letter('q')
r = Letter('r')

f = Imp(Eq(p,q),And(Not(p), r))

print('DIMACS:')
dimacs_f = dimacs(f)
print(dimacs_f)

print()
print()
print('DPLL')
DPLL(dimacs_f, {}, True)

DIMACS:
[[2, 3, -2], [2, 3, 1], [2, -2, -2], [2, -2, 1], [-3, 3, -2], [-3, 3, 1], [-3, -2, -2], [-3, -2, 1]]


DPLL
TAUTOLOGY:
[[2, 3, 1], [-3, -2, -2], [-3, -2, 1]]
[1, 2, 3, -3, -2]
PURE LITERAL:
[[2, 3, 'T'], [-3, -2, -2], [-3, -2, 'T']]
TAUTOLOGY:
[[-3, -2, -2]]
[-3, -2]
PURE LITERAL:
[['-F', -2, -2]]
NEGATION SUBSTITUTION:
[['T', -2, -2]]
TAUTOLOGY:
[]


(True, {1: True, 3: False})

c) Napisati funkciju koja koriscenjem funkcije _DPLL_ ispituje da li je formula **valjana**.

In [71]:
def valjana(formula):
    not_formula = Not(formula)
    
    D = dimacs(not_formula)
    rezult, val = DPLL(D, {}, False)
    if rezult:
        print('Formula nije valjana.')
    else:
        print('Formula je valjana.')

d) Pozvati napisanu funkciju za formulu:

$(p \land (\neg p \lor q) \land (p \lor \neg q)) \Rightarrow (p \land q)$

In [72]:
formula2 = Imp(And(p, And(Or(Not(p), q),Or(p, Not(q)))), And(p, q))

In [73]:
valjana(formula2)

Formula je valjana.
